# Install Necessary Packages

In [ ]:
!pip install transformers[torch]

  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached accelerate-1.4.0-py3-none-any.whl.metadata (19 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached accelerate-1.4.0-py3-none-any.whl (342 kB)
Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl (308 kB)
Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl (2.4 MB)
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 1.3/204.1 MB 9.6 MB/s eta 0:00:22
    --------------------------------------- 3.7/204.1 MB 10.4 MB/s eta 0:00:20
   - -------------------------------------- 6.0/204.1 MB 10.5 MB/s eta 0:00:19
   - -------------------------------------- 8.4/204.1 MB 10.6 MB/s eta 0:00:19
   -- -------

In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install datasets

   ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
   --- ------------------------------------ 2.4/25.3 MB 11.2 MB/s eta 0:00:03
   ------- -------------------------------- 4.5/25.3 MB 11.2 MB/s eta 0:00:02
   ---------- ----------------------------- 6.8/25.3 MB 11.0 MB/s eta 0:00:02
   -------------- ------------------------- 9.2/25.3 MB 11.2 MB/s eta 0:00:02
   ------------------ --------------------- 11.5/25.3 MB 11.3 MB/s eta 0:00:02
   ---------------------- ----------------- 13.9/25.3 MB 11.2 MB/s eta 0:00:02
   ------------------------- -------------- 16.0/25.3 MB 11.2 MB/s eta 0:00:01
   ----------------------------- ---------- 18.4/25.3 MB 11.2 MB/s eta 0:00:01
   -------------------------------- ------- 20.7/25.3 MB 11.2 MB/s eta 0:00:01
   ------------------------------------ --- 23.1/25.3 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------  25.2/25.3 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 25.3/25.3 MB 11.0 MB/

In [ ]:
from sentence_transformers import SentenceTransformer
import torch
from datasets import load_dataset, Dataset


#   Load Dataset

More infromation about the dataset over here: # https://sbert.net/docs/sentence_transformer/training_overview.html#dataset


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv("df_entrenamiento3.csv")

# Dividir el DataFrame en train+eval y test
train_eval_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Dividir train+eval en train y eval
train_data, eval_data = train_test_split(train_eval_data, test_size=0.2, random_state=42)

# Convertir los DataFrames a Datasets de Hugging Face
train_dataset = Dataset.from_pandas(train_data)
eval_dataset = Dataset.from_pandas(eval_data)
test_dataset = Dataset.from_pandas(test_data)

# Verificar el tamaño de los conjuntos
print(f"Tamaño del conjunto de entrenamiento: {len(train_dataset)}")
print(f"Tamaño del conjunto de evaluación: {len(eval_dataset)}")
print(f"Tamaño del conjunto de prueba: {len(test_dataset)}")

In [ ]:
train_dataset = train_dataset.remove_columns(['__index_level_0__'])
test_dataset = test_dataset.remove_columns(['__index_level_0__'])
eval_dataset = eval_dataset.remove_columns(['__index_level_0__'])

# Load Model

In [ ]:
# 1. Load a model to finetune with 2. (Optional) model card data

#popular embedding models:
#https://huggingface.co/nomic-ai/nomic-embed-text-v1
#https://huggingface.co/BAAI/bge-large-en

model = SentenceTransformer("sentence-transformers/all-distilroberta-v1")

C:\Users\danip\anaconda3\envs\recomendador\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\danip\.cache\huggingface\hub\models--sentence-transformers--all-distilroberta-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


# Setting up Training Arguments

In [ ]:
from sentence_transformers import SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from sentence_transformers.losses import CosineSimilarityLoss, CoSENTLoss
from sentence_transformers.training_args import BatchSamplers



In [ ]:
# 3. Define a loss function
#loss = CosineSimilarityLoss(model)
loss = CoSENTLoss(model)


In [ ]:
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/fine-tune-1",
    # Optional training parameters:
    num_train_epochs=1,
    per_device_train_batch_size=160,
    per_device_eval_batch_size=160,
    warmup_ratio=0.1,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=10,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    run_name="mpnet-base-all-nli-triplet",  # Will be used in W&B if `wandb` is installed
)

# Train

In [ ]:
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss
)


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
10,No log,3.631134
20,No log,3.142216
30,No log,3.196457
40,No log,3.332477
50,No log,3.454187
60,No log,3.599512
70,No log,3.780658
80,No log,3.947933
90,No log,3.614004


# Test - Model any good?

In [ ]:

from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction
# Initialize the evaluator
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=eval_dataset["sentence1"],
    sentences2=eval_dataset["sentence2"],
    scores=eval_dataset["score"],
    name="sts_dev",
)
results = dev_evaluator(model)
'''
EmbeddingSimilarityEvaluator: Evaluating the model on the sts-dev dataset:
Cosine-Similarity :  Pearson: 0.8806 Spearman: 0.8810
'''
print(dev_evaluator.primary_metric)
# => "sts_dev_pearson_cosine"
print(results[dev_evaluator.primary_metric])
# => 0.881019449484294

sts_dev_spearman_cosine
0.7853352738169416


In [ ]:

# 8. Save the trained model
model.save_pretrained("models/all-distilroberta-v1/version3")

# 9. (Optional) Push it to the Hugging Face Hub
#model.push_to_hub("mpnet-base-all-nli-triplet")